# Demonstration of EFlowCalc's API

This notebook contains a simple example of the usage of the API of `EFlowCalc` to calculate stream flow characteristics from stream flow time series.

## 1. Load the streamflow timeseries

An example file is provided in the folder `examples/` in order for anyone to reproduce this tutorial. Because this is a NetCDF file, we are going to use the Python package `netCDF4`, but EFlowCalc is independent of the file format you are working with because it only requires numpy arrays as inputs for streamflow timeseries and datetime series. The datetime series must be made of `datetime` objects from the `datetime` package.

In [ ]:
from netCDF4 import Dataset
import numpy as np
from datetime import datetime, timedelta

# load the timestamp and streamflow timeseries
with Dataset('examples/catchment.sim.flow.nc', 'r', format='NETCDF4') as f:  # read the NetCDF file
    my_flow = f.variables['flow'][:]  # these are the streamflow timeseries
    my_ts = f.variables['DateTime'][:]  # this is the timestamp series for the period

# convert the UNIX Epoch time (contained in the NetCDF file) to datetime objects
my_dt = np.array([datetime(1970, 1, 1) + timedelta(seconds=tstamp) for tstamp in my_ts]) 

## 2. Calculate one or more streamflow charateristics

Now that the data set is loaded in memory, it is time to use EFlowCalc to calculate the stream flow characteristics from the hydrograph(s). To do so, import `eflowcalc`, which will give you access to the `calculator` Python function as well as all stream flow characteristics implemented in EFlowCalc (as Python functions as well). To calculate any characteristic from your stream flow time series, you only need to call the calculator function and provide four positional arguments: 
* the Python function (e.g. `ma41`, `dh4`, etc.) or an iterable containing several functions (e.g. `(ma41, dh4)`, `[ml1, mh1, ra7]`, etc.) corresponding to the characteristic(s) to be calculated;
* the numpy array containing the DateTime that cover the period the stream flows are provided for (it can only contain one time series); 
* the numpy array containing the stream flow time series (it can be multiple time series); 
* the catchment drainage area in square kilometers. 

See below examples using these requirements:

In [ ]:
from eflowcalc import calculator, ma41, dh4, ra7

# calculate only one streamflow characteristic (e.g. MA41 here)
my_ma41 = calculator(ma41, my_dt, my_flow, 1246)

# calculate multiple streamflow characteristics at once (e.g. MA41, DH4, and RA7 here)
my_sfc = calculator((ma41, dh4, ra7), my_dt, my_flow, 1246)

It is important to be aware that EFlowCalc requires strictly continuous time series of daily stream flow. Moreover, all stream flow characteristics are only computed on full hydrological years (it will automatically trim the head and tail of the time series to guarantee so). By default, a hydrological year starts on the 1st of October, which is commonly the case in the Northern hemisphere. However, it can be changed to suit any location using the keyword argument `hydro_year`. For example, if working on a catchment in the Southern hemisphere, it is likely that the hydrological year starts on the 1st of July:

In [ ]:
# calculate only one streamflow characteristic (e.g. MA41 here) for a catchment in the Southern hemisphere
my_ma41 = calculator(ma41, my_dt, my_flow, 1246, hydro_year='01/07')

EFlowCalc can only deal with 1-dimensional or 2-dimensional numpy arrays as input. EFlowCalc can evaluate multiple simulated timeseries at the same time (using vectorised calculations). This means that the stream flow numpy array can be 2-dimensional, one of them being the time dimension and must match the length of the DateTime numpy array length. By default, EFlowCalc is expecting the time dimension to be on the first axis (i.e. `axis=0`) if a 2-dimensional array is provided (applicable to both the stream flow and datetime arrays). If this is not the case with your data set, you need to redefine the `axis` keyword argument default value and set it to `1` (for the second axis). Alternatively, you can transpose your arrays before giving them to EFlowCalc.